In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [3]:
df = pd.read_csv('2020.12.13 boiling_plan.csv')
df['sku'] = df['sku'].apply(cast_sku)
df['boiling'] = df['sku'].apply(lambda sku: sku.boilings[0])
df.sort_values(by='id')
df['packing_speed'] = df['sku'].apply(lambda sku: sku.packing_speed)
df['boiling_form_factor'] = df['sku'].apply(lambda sku:sku.boiling_form_factors[0])

df.at[:2, 'packing_team_id'] = 1
df.at[2:, 'packing_team_id'] = 2

# mark batch
cur_batch = 0
cur_boiling = None
batch_values = []
for i, row in df.iterrows():
    if row['boiling'] != cur_boiling:
        cur_batch += 1
        cur_boiling = row['boiling']
    batch_values.append(cur_batch)
df['batch'] = batch_values

boiling_plan_df = df.copy()
boiling_plan_df.head()

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id,batch
0,1.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0,2
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0,2
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0,2


In [4]:
def gen_boiling_plan(boiling_plan_df, id):
    boiling_plan = boiling_plan_df[boiling_plan_df['id'] == id].copy()

    # mark groups
    cur_group = 0
    cur_form_factor = None
    group_values = []
    for i, row in boiling_plan.iterrows():
        if row['boiling_form_factor'] != cur_form_factor:
            cur_group += 1
            cur_form_factor = row['boiling_form_factor']
        group_values.append(cur_group)
    boiling_plan['group'] = group_values
    return boiling_plan

# Create schedule for sample boiling

In [5]:
boiling_plan = gen_boiling_plan(boiling_plan_df, 1)
boiling_model = boiling_plan.iloc[0]['boiling']

# todo: del
if boiling_model.boiling_type == 'salt':
    # todo: make properly
    boiling_model.meltings.speed = 850 / 50 * 60

maker, root, make = new_block_maker()

# [termizator.time]
# todo: del
termizator = db.session.query(Termizator).first()
termizator.pouring_time = 30

with make('boiling', boiling_model=boiling_model, boiling_plan=boiling_plan):
    with make('pouring'):
        with make('first'):
            make('termizator', time_size=termizator.pouring_time)
            make('fermenting', time_size=boiling_model.pourings.pouring_time - termizator.pouring_time)
            make('soldification', time_size=boiling_model.pourings.soldification_time)
            make('cutting', time_size=boiling_model.pourings.cutting_time)
        with make('second'):
            make('pouring_off', time_size=boiling_model.pourings.pouring_off_time)
            make('extra', time_size=boiling_model.pourings.extra_time)
            
root

root (0, 1] x (0, 27]
  boiling (0, 1] x (0, 27]
    pouring (0, 1] x (0, 27]
      first (0, 1] x (0, 22]
        termizator (0, 1] x (0, 6]
        fermenting (0, 1] x (6, 8]
        soldification (0, 1] x (8, 15]
        cutting (0, 1] x (15, 22]
      second (0, 1] x (22, 27]
        pouring_off (0, 1] x (22, 25]
        extra (0, 1] x (25, 27]

In [6]:
def make_boiling(boiling_plan):
    boiling_model = boiling_plan.iloc[0]['boiling']
    
    # todo: del
    if boiling_model.boiling_type == 'salt':
        # todo: make properly
        boiling_model.meltings.speed = 850 / 50 * 60

    maker, root, make = new_block_maker()

    # [termizator.time]
    # todo: del
    termizator = db.session.query(Termizator).first()
    termizator.pouring_time = 30
    
    with make('boiling', boiling_model=boiling_model, boiling_plan=boiling_plan):
        with make('pouring'):
            with make('first'):
                make('termizator', time_size=termizator.pouring_time)
                make('fermenting', time_size=boiling_model.pourings.pouring_time - termizator.pouring_time)
                make('soldification', time_size=boiling_model.pourings.soldification_time)
                make('cutting', time_size=boiling_model.pourings.cutting_time)
            with make('second'):
                make('pouring_off', time_size=boiling_model.pourings.pouring_off_time)
                make('extra', time_size=boiling_model.pourings.extra_time)

        make('drenator', 
             t=root['boiling']['pouring']['first'].end(),
             time_size=boiling_model.lines.cheddarization_time * 12 * 5, 
             push_func='add')

        with make('melting'):
            serving = make('serving', push_func='add', time_size=boiling_model.meltings.serving_time).block

            meltings = make('meltings', push_func='add', t=serving.length()).block
            coolings = make('coolings', push_func='add', t=serving.length()).block

            for group, grp in boiling_plan.groupby('group'):
                if group > 1 and boiling_model.boiling_type == 'water':
                    # non-first group - reconfigure time
                    push(meltings, Block('melting_configuration', size=1), 'dummy')

                melting_process = Block('melting_process',
                                        time_size=custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil'),
                                       actual_time_size=grp['kg'].sum() / boiling_model.meltings.speed * 60,
                                       boiling_form_factor=grp.iloc[0]['boiling_form_factor'])

                push(meltings, melting_process, 'dummy')

                # add cooling
                cooling_process = Block('cooling_process', t=melting_process.props.relative_props['t'])
                cooling_maker, _, cooling_make = new_block_maker(cooling_process)

                with cooling_make('start'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=boiling_model.meltings.first_cooling_time)
                        cooling_make('cooling', time_size=boiling_model.meltings.second_cooling_time)
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=boiling_model.meltings.salting_time)

                with cooling_make('finish'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=melting_process.props['time_size'])
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=melting_process.props['time_size'])

                push(coolings, cooling_process, 'add')
                
    return root['boiling']

In [7]:
boilings = []
ids = list(boiling_plan_df['id'].unique())[:2]
for id in ids:
    boiling_plan = gen_boiling_plan(boiling_plan_df, id)
    boiling = make_boiling(boiling_plan)
    print('Boiling', id, boiling.props['boiling_model'].boiling_type)
    print(draw_print(boiling))
    boilings.append(boiling)

Boiling 1.0 water
================================================================================================== boiling 0:[0,98)
=========================== pouring 0:[0,27)
====================== first 0:[0,22)
====== termizator 0:[0,6)
      == fermenting 0:[6,8)
        ======= soldification 0:[8,15)
               ======= cutting 0:[15,22)
                      ===== second 0:[22,27)
                      === pouring_off 0:[22,25)
                         == extra 0:[25,27)
                      ================================================ drenator 0:[22,70)
                                                                      ============================ melting 0:[70,98)
                                                                      ====== serving 0:[70,76)
                                                                            ============== meltings 0:[76,90)
                                                                            ============== melting_p

In [8]:
def gen_label(label_weights):
    cur_label = None
    values = []
    for label, weight in label_weights:
        s = ''
        if label != cur_label:
            s += label + ' '
            cur_label = label

        # todo: make properly, this if-else if because of rubber. Should be something better
        if weight:
            s += str(weight / 1000)
        else:
            s += 'терка'
        values.append(s)
    return '/'.join(values)

def gen_basic_packing(boiling):
    boiling_plan = boiling.props['boiling_plan']
    boiling_model = boiling.props['boiling_model']
    
    brand_label = gen_label([(sku.brand_name, sku.weight_form_factor) for sku in boiling_plan['sku'].values])

    maker, packing, make = new_block_maker(root='packing', default_push_func=dummy_push)
    packing.props.update({'t': listify(boiling['melting']['coolings']['cooling_process'])[0]['start'].end(), 
                          'brand_label': brand_label})

    for i, (_, row) in enumerate(boiling_plan.iterrows()):
        sku, kg = row['sku'], row['kg']
        packing_speed = min(sku.packing_speed, boiling_model.meltings.speed)
        make('packing_process', time_size=custom_round(kg / packing_speed * 60, 5, rounding='ceil'))

        if i != len(boiling_plan) - 1:
            # add configuration
            new_sku = boiling_plan.iloc[i + 1]['sku']
            conf_time_size = 0
            if boiling_model.boiling_type == 'salt' and sku.weight_form_factor != new_sku.weight_form_factor and cast_packer(sku) == cast_packer(new_sku):
                # todo: take from parameters
                conf_time_size = 25
            elif sku == new_sku:
                conf_time_size = 0
            else:
                # todo: take from parameters
                conf_time_size = 5
            
            if conf_time_size:
                make('configuration', time_size=conf_time_size)
                
    return packing

In [9]:
packing = gen_basic_packing(boiling)
packing

packing (0, 1] x (84, 98]
  packing_process (0, 1] x (84, 98]

# Make Schedule

In [10]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return

    boilings = [node for node in parent.children if node.props['class'] == 'boiling']

    b2 = boiling

    for b1 in reversed(boilings[-6:]): # compare with four previous boilings
        validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])  # [termizator.basic]

        # cannot make two boilings on same line at the same time
        if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
            validate_disjoint(b1['pouring'], b2['pouring'])

        if b1.props['boiling_type'] == b2.props['boiling_type']:
            # [melting.disjoint]
            validate_disjoint(b1['melting']['meltings'], b2['melting']['meltings'])
            
        validate_disjoint(b1['packing'], b2['packing'])

In [11]:
def generate_schedule_basic():
    lines_df = pd.DataFrame(index=['water', 'salt'], columns=['iter_props', 'start_time', 'boilings_left', 'latest_boiling'])
    lines_df.at['water', 'iter_props'] = [{'pouring_line': str(v)} for v in [0, 1]]
    lines_df.at['salt', 'iter_props'] = [{'pouring_line': str(v)} for v in [2, 3]]

    # [cheesemakers.start_time]
    lines_df.at['water', 'start_time'] = '09:50'
    lines_df.at['salt', 'start_time'] = '07:05'

    # generate boilings
    for boiling_type in ['water', 'salt']:
        ids = boiling_plan_df[boiling_plan_df['boiling'].apply(lambda b: b.boiling_type) == boiling_type].id
        ids = list(ids)
        ids = remove_duplicates(ids)
        
        boilings = []
        for id in ids:
            boiling = make_boiling(gen_boiling_plan(boiling_plan_df, id))
            packing = gen_basic_packing(boiling)
            add_push(boiling, packing)
            boiling.props.update({'block_front_id': int(id)})
            boilings.append(boiling)
        
        lines_df.at[boiling_type, 'boilings_left'] = boilings

    lines_df['latest_boiling'] = None

    schedule = Block('schedule')

    def add_block(boiling_type):
        boiling = lines_df.at[boiling_type, 'boilings_left'].pop(0)

        if not lines_df.at[boiling_type, 'latest_boiling']:
            beg = cast_t(lines_df.at[boiling_type, 'start_time']) - boiling['melting'].beg()
        else:
            beg = lines_df.at[boiling_type, 'latest_boiling'].x1

        dummy_push(schedule, boiling, iter_props=lines_df.at[boiling_type, 'iter_props'], validator=boiling_validator, beg=beg, max_tries=100)

        lines_df.at[boiling_type, 'latest_boiling'] = boiling
        return boiling
    
    add_block('water')
    add_block('salt')
    
    while True:
        are_boilings_left = lines_df['boilings_left'].apply(lambda lst: len(lst) > 0)
        if are_boilings_left.sum() == 0:
            # finished
            break
        elif are_boilings_left.sum() == 1:
            boiling_type = are_boilings_left[are_boilings_left].index[0]
        elif are_boilings_left.sum() == 2:
            boiling_type = max(lines_df['latest_boiling'], key=lambda b: b.beg()).props['boiling_model'].boiling_type
            # reverse
            boiling_type = 'water' if boiling_type == 'salt' else 'salt'
        else:
            raise Exception('Should not happen')
            
        add_block(boiling_type)
        
    return schedule

In [12]:
schedule = generate_schedule_basic()

In [13]:
schedule

schedule (0, 1] x (0, 426]
  boiling (0, 1] x (48, 146]
    pouring (0, 1] x (48, 75]
      first (0, 1] x (48, 70]
        termizator (0, 1] x (48, 54]
        fermenting (0, 1] x (54, 56]
        soldification (0, 1] x (56, 63]
        cutting (0, 1] x (63, 70]
      second (0, 1] x (70, 75]
        pouring_off (0, 1] x (70, 73]
        extra (0, 1] x (73, 75]
    drenator (0, 1] x (70, 118]
    melting (0, 1] x (118, 146]
      serving (0, 1] x (118, 124]
      meltings (0, 1] x (124, 138]
        melting_process (0, 1] x (124, 138]
      coolings (0, 1] x (124, 146]
        cooling_process (0, 1] x (124, 146]
          start (0, 1] x (124, 132]
            cooling (0, 1] x (124, 128]
            cooling (0, 1] x (128, 132]
          finish (0, 1] x (132, 146]
            cooling (0, 1] x (132, 146]
    packing (0, 1] x (132, 146]
      packing_process (0, 1] x (132, 146]
  boiling (0, 1] x (29, 119]
    pouring (0, 1] x (29, 54]
      first (0, 1] x (29, 49]
        termizator (0, 

# Draw boilings

In [27]:
def add_push_with_props(parent, block, **kwargs):
    new_props = block.props.get_all_props()
    new_props.update(kwargs)
    return add_push(parent, block.clone(), new_props=new_props)


def gen_header(start_time='01:00'):
    maker, front, make = new_block_maker('header', default_push_func=dummy_push)
    
    with make('header', h=1, index_width=3):
        make(h=1, size=1, text='График наливов', push_func=add_push)
        for i in range(288):
            cur_time = cast_time(i + cast_t(start_time))
            if cur_time[-2:] == '00':
                make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    return front['header']


def gen_simple_frontend(schedule):
    maker, front, make = new_block_maker('front')

    for i, boiling in enumerate(listify(schedule['boiling'])):
        with make('boiling', axis='y', block_front_id=f'{i} {boiling.props["pouring_line"]}'):
            make(boiling['pouring'], push_func=add_push_with_props, push_kwargs={'y': 0})
            make(boiling['drenator'], push_func=add_push_with_props, push_kwargs={'y': 1})
            make(boiling['melting']['serving'], push_func=add_push_with_props, push_kwargs={'y': 1})
            make(boiling['melting']['meltings'], push_func=add_push_with_props, push_kwargs={'y': 1})

            n_cooling_lines = 4
            cooling_lines = [make('cooling_line', h=1, axis='y').block for i in range(n_cooling_lines)]
            cur_i = 0
            for cooling_process in listify(boiling['melting']['coolings']['cooling_process']):
                for cooling in cooling_process.iter():
                    if cooling.props['class'] in ['cooling', 'salting']:
                        add_push_with_props(cooling_lines[cur_i % n_cooling_lines], cooling)
                cur_i += 1
            
            with make('packing', axis='y'):
                make(boiling['packing'], push_func=add_push_with_props)
    return front

def gen_cheese_makers(schedule, rng):
    maker, front, make = new_block_maker('cheese_makers')

    for i in rng:
        with make(f'cheese_maker', h=2, axis='y'):
            for boiling in schedule.iter({'class': 'boiling', 'pouring_line': str(i)}):
                boiling_model = boiling.props['boiling_model']
                # todo: make properly
                boiling_volume = 8000

                # [cheesemakers.boiling_params]
                boiling_label = '{} {} {} {}кг'.format(boiling_model.percent, boiling_model.ferment, '' if boiling_model.is_lactose else 'безлактозная', boiling_volume)

                with make('pouring_block', push_func='add'):
                    with make(h=1, axis='y'):
                        make(boiling['pouring']['first']['termizator'], push_func=add_push_with_props)
                        make('pouring_name', size=boiling['pouring'].length() - boiling['pouring']['first']['termizator'].length(), boiling_label=boiling_label)
                    with make(h=1, axis='y'):
                        make('pouring_and_fermenting', t=boiling['pouring']['first']['termizator'].beg(), 
                             size=boiling['pouring']['first']['termizator'].length() + boiling['pouring']['first']['fermenting'].length(),
                            axis='y')
                        make(boiling['pouring']['first']['soldification'], push_func=add_push_with_props)
                        make(boiling['pouring']['first']['cutting'], push_func=add_push_with_props)
                        make(boiling['pouring']['second']['pouring_off'], push_func=add_push_with_props)
                        make(boiling['pouring']['second']['extra'], push_func=add_push_with_props)
        make(h=1, visible=False, axis='y')
    return front

In [28]:
def draw_excel_frontend(schedule):
    maker, front, make = new_block_maker('front')

    make('stub', h=1, axis='y')
    make(gen_header(), axis='y')
#     make(gen_simple_frontend(schedule), axis='y')
    make(gen_cheese_makers(schedule, range(2)), axis='y')
    
    style = load_style()
    wb = draw_schedule(front, style, init_sheet_func=init_sheet)
    sf = SplitFile('schedule.xlsx')
    fn = sf.get_new()
    wb.save(fn)
    open_file_in_os(fn)
    return front

In [29]:
front = draw_excel_frontend(schedule)

In [22]:
gen_cheese_makers(axis='y')

TypeError: gen_cheese_makers() got an unexpected keyword argument 'axis'